# Data Preprocessing
get the user item matrix from the raw data

In [2]:
import pandas as pd

df_rating = pd.read_csv('datasets/ratings.csv')
df_item = pd.read_csv('datasets/movies.csv')

df_user_item = df_rating.pivot(index='userId', columns='movieId', values='rating')
df_user_item = df_user_item.fillna(0)

df_user_item.to_csv('datasets/user_item.csv')

In [2]:
import pandas as pd

# Leggi il file CSV
df_links = pd.read_csv('datasets/links.csv')

# Controlla i valori nella colonna imdbId
print(df_links['imdbId'].unique())

# Rimuovi gli zeri iniziali solo per i valori validi
df_links['imdbId'] = df_links['imdbId'].apply(lambda x: str(x).lstrip('0') if pd.notnull(x) else x)

# Salva il DataFrame pulito nel file CSV
df_links.to_csv('datasets/links.csv', index=False)


[ 114709  113497  113228 ... 6397426 8391976  101726]


In [7]:
import pandas as pd

# Load data from CSV files
df_links = pd.read_csv('datasets/links.csv')
df_movies_links = pd.read_csv('datasets/movies_links.csv')

# Extract unique imdbId and tmdbId from both DataFrames
imdbId_links = df_links['imdbId'].unique()
imdbId_movies_links = df_movies_links['imdbId'].unique()

# Check if all imdbId in df_links are present in df_movies_links
missing_imdb_ids = [x for x in imdbId_links if x not in imdbId_movies_links]
all_present = len(missing_imdb_ids) == 0

# Check if each imdbId in df_movies_links appears only once in df_links
unique_in_movies_links = df_movies_links['imdbId'].value_counts() == 1
all_unique = all(unique_in_movies_links)

# Check tmdbId correspondence
tmdb_mismatch = []
for i, row in df_links.iterrows():
    imdb_id = row['imdbId']
    tmdb_id_link = row['tmdbId']
    
    # Find matching row in df_movies_links based on imdbId
    matched_row = df_movies_links[df_movies_links['imdbId'] == imdb_id]
    if not matched_row.empty:
        tmdb_id_movies = matched_row['tmdbId'].iloc[0]
        
        # Check if both tmdbIds are null or they match
        if (pd.isna(tmdb_id_link) and pd.isna(tmdb_id_movies)) or (tmdb_id_link == tmdb_id_movies):
            continue
        else:
            tmdb_mismatch.append(imdb_id)

# Final result
if all_present and all_unique and not tmdb_mismatch:
    print("All imdbId from df_links are present exactly once in df_movies_links, with matching tmdbId values.")
else:
    if missing_imdb_ids:
        print("The following imdbIds from df_links are missing in df_movies_links:", missing_imdb_ids)
    if not all_unique:
        print("Some imdbIds in df_movies_links are not unique.")
    if tmdb_mismatch:
        print("The following imdbIds have mismatching tmdbId values:", tmdb_mismatch)


All imdbId from df_links are present exactly once in df_movies_links, with matching tmdbId values.


In [4]:
# cast tmdbId to int in links.csv and movies_links.csv
df_links['tmdbId'] = df_links['tmdbId'].astype('Int64')

df_movies_links['tmdbId'] = df_movies_links['tmdbId'].astype('Int64')

# Save the cleaned DataFrames to CSV files
df_links.to_csv('datasets/links.csv', index=False)
df_movies_links.to_csv('datasets/movies_links.csv', index=False)


In [23]:
#add movieID present in links.csv to movie_details.csv and rename the column to movieLensId
#join the two dataframes on imdb_id (on movie_details.csv) and imdbId (on links.csv)

import pandas as pd

# Load data from CSV files
df_movie_details = pd.read_csv('datasets/movie_details.csv')
df_links = pd.read_csv('datasets/links.csv')

#remove the NaN values in imdb_id
df_movie_details = df_movie_details.dropna(subset=['imdb_id'])

#convert imdbId to int
df_links['imdbId'] = df_links['imdbId'].astype(int)
df_movie_details['imdb_id'] = df_movie_details['imdb_id'].str.extract('(\d+)').astype(int)

# Rename the column to movieLensId
df_links = df_links.rename(columns={'movieId': 'movieLensId'})

# Join the two DataFrames on imdb_id and imdbId
df_movie_details = df_movie_details.merge(df_links, how='inner', left_on='imdb_id', right_on='imdbId')

#remove the columns imdb_id in movie_details.csv
df_movie_details = df_movie_details.drop(columns=['imdb_id'])

#remove the id column
df_movie_details = df_movie_details.drop(columns=['id'])

#convert tmdbId to int
df_movie_details['tmdbId'] = df_movie_details['tmdbId'].astype(int)

# Save the cleaned DataFrame to a CSV file
df_movie_details.to_csv('datasets/movie_details.csv', index=False)




